# Initial Flyback Calculations (LTSpice)

In [2]:
import math

In [8]:
# Input Variables
v_in = 12 # Input voltage (V)
v_out = 2000 # Output voltage (V)
i_out = 0.005 # Output current (A)
np = 10 # Transformer primary turns 
ns = 1350 # Transformer secondary turns
lp = 19*10**(-6) # Transformer primary inductance (datasheet) (Henry's)
ls = 1 # Transformer secondary (datasheet) (Herny's)
lr_p = 0.120 # Transformer primary resistance (Ohm's)
lr_s = 8100 # Transformer secondary resistance (Ohm's)

# Calculations
nps = ns/np # primary-to-secondary turns ratio

In [9]:
# Calculate the secondary inductance ratio given the turns ratio to properly model the transformer
ls_calc = lp/(np/ns)**2
print("Secondary Inductance Ls: %.3f H" % ls_calc) 

Secondary Inductance Ls: 0.346 H


Why doesn't Ls_calc match the datasheet?... LTSpice initial simulation stalls out with 0V output after a quick spike with 1H secondary inductance...

# LT3748 Calculations

In [15]:
# Input Variable
v_df = 1.1 # Forward voltage rating (V)
v_dr = 1000 # Reverse voltage rating (V)
rref = 6040 # LT3748 datasheet given (Ohm's)
v_tc = 0.55 # LT3748 datasheet given (V)


# Calculations
df_cnt = math.ceil((v_out*2)/v_dr) # 50% margin
# Calculate diode leakage safety resistors (http://engineering.electrical-equipment.org/electrical-distribution/series-and-parallel-connected-diodes.html)
v_bg_typ = 1.223 # LT3748 internal bandgap voltage referernce typical voltage (V)
v_df_total = v_df*df_cnt

In [19]:
# LT3748 Feedback resistor
r_fb = (rref*(1/nps)*((v_out+v_df_total)+v_tc))/(v_bg_typ)

print("R_FB = %d Ohms" % r_fb)

R_FB = 73346 Ohms
